In [26]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb


In [27]:
review_len = 250
(train_reviews, train_labels), (test_reviews, test_labels) = imdb.load_data(maxlen = review_len)
(train_reviews_bak, train_labels_bak), (test_reviews_bak, test_labels_bak) = imdb.load_data(maxlen = review_len)

In [28]:
train_reviews = pad_sequences(train_reviews, padding='post')
test_reviews = pad_sequences(test_reviews, padding='post')
train_labels = keras.utils.to_categorical(train_labels, 2)
test_labels = keras.utils.to_categorical(test_labels, 2)


In [29]:
index = 0
print(train_reviews[index])
print(train_labels[index])


[    1    14    22    16    43   530   973  1622  1385    65   458  4468
    66  3941     4   173    36   256     5    25   100    43   838   112
    50   670 22665     9    35   480   284     5   150     4   172   112
   167 21631   336   385    39     4   172  4536  1111    17   546    38
    13   447     4   192    50    16     6   147  2025    19    14    22
     4  1920  4613   469     4    22    71    87    12    16    43   530
    38    76    15    13  1247     4    22    17   515    17    12    16
   626    18 19193     5    62   386    12     8   316     8   106     5
     4  2223  5244    16   480    66  3785    33     4   130    12    16
    38   619     5    25   124    51    36   135    48    25  1415    33
     6    22    12   215    28    77    52     5    14   407    16    82
 10311     8     4   107   117  5952    15   256     4 31050     7  3766
     5   723    36    71    43   530   476    26   400   317    46     7
     4 12118  1029    13   104    88     4   381   

In [30]:
word_index = imdb.get_word_index()
#inverted_word_index = dict((i, word) for (word, i) in word_index)

In [31]:
word_index['great']

84

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [33]:
model = Sequential()
vocab_size = len(word_index)
embedding_dim = 16
max_size = review_len -1
model.add(Embedding(vocab_size, embedding_dim, input_length = max_size))
model.add(GlobalAveragePooling1D())

model.add(Dense(14, activation = 'sigmoid'))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 249, 16)           1417344   
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 14)                238       
                                                                 
 dense_1 (Dense)             (None, 12)                180       
                                                                 
 dense_2 (Dense)             (None, 2)                 26        
                                                                 
Total params: 1,417,788
Trainable params: 1,417,788
Non-trainable params: 0
______________________________________________

In [34]:
model.fit(train_reviews, train_labels, epochs=10, shuffle = True)


Epoch 1/10
536/536 [==============================] - 48s 87ms/step - loss: 0.6803 - accuracy: 0.5857ETA: 27s - loss: 0.6930 - a - ETA: 26s - loss: 0.6928 - accuracy: 0 - ETA: 25s - loss: -  - ETA: 7s - loss: 0.6869 - accura - ETA: 6s - l -
Epoch 2/10
536/536 [==============================] - 48s 89ms/step - loss: 0.3994 - accuracy: 0.8516
Epoch 3/10
536/536 [==============================] - 47s 88ms/step - loss: 0.2302 - accuracy: 0.9105: 35s - loss: 0.2544 - ac - ETA: 35s - loss: 0.2497 - ac
Epoch 4/10
536/536 [==============================] - 47s 88ms/step - loss: 0.1735 - accuracy: 0.9354
Epoch 5/10
536/536 [==============================] - 48s 89ms/step - loss: 0.1343 - accuracy: 0.9530: 21s - loss: 0.1299 - accuracy: 0.95 - ETA: 21s - loss: 0.13 - ETA: 1s - l
Epoch 6/10
536/536 [==============================] - 49s 91ms/step - loss: 0.1029 - accuracy: 0.9662: 45s - loss: 0.1 - ETA: 37s - loss: 0.1 - ETA: 32s - loss: 0.1038 - - ETA:  - ETA:  - ETA: 15s - loss: 0.1054 - accura

In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [36]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token='<OOV>')
tokenizer.word_index = word_index

In [37]:
sentences = [
     'hi'
    
 ]
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post', maxlen = review_len -1)

In [38]:
index = 0
sample = padded[index].reshape(1, len(padded[index]))
model.predict(sample)

array([[0.28888386, 0.7111162 ]], dtype=float32)

In [39]:
expected = np.asarray([[0, 1]])

In [40]:
model.fit(padded, expected, epochs=1)

1/1 [==============================] - 0s 490ms/step - loss: 0.4931 - accuracy: 1.0000


In [41]:
model.save('nlp.h5')